In [1]:
import csv
import itertools
import pyreadstat

In [2]:
def spss_to_singularity(filename):
    """
    Converting SPSS files to datamap and dataset for use on Singularity.
    Reminder to open the output files in Excel and do the following:
        (1) Save the files within Excel to convert floats to ints for the dataset
        (2) Confirm the encoding is CSV, specifically (i.e. not UTF-8)
    """

    
    pn = filename.split(".",1)[0]
    df, meta = pyreadstat.read_sav(filename, encoding="latin1")
    
    #---------------------------------------------
    # Compiling Datamap
    #---------------------------------------------
    dmap_rows = []
    for q in meta.column_names:
        row_0, row_1 = [], []

        row_0 = [q, 'integer', '{}', meta.column_names_to_labels.get(q).split(': ',1)[1]] # variable labels

        try: row_1 = list(itertools.chain(*list(meta.value_labels.get(meta.variable_to_label.get(q)).items()))) # value labels
        except: pass # not all vars have value labels
        for i in range(len(row_1)):
            if (i%2)==0: row_1[i] = int(row_1[i]) # converting floats to ints so Singularity can read it

        dmap_rows.append(row_0 + row_1) #add them all together
        
    #---------------------------------------------
    # Creating Output Files
    #---------------------------------------------
    # Datamap
    with open(f'{pn}_datamap.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(dmap_rows)
        
    # Dataset
    df.to_csv(f'{pn}_dataset.csv',index=False)
    
    return

In [3]:
spss_to_singularity('2108140.sav')